In [8]:
import numpy as np
from itertools import islice


In [9]:
data_ = {}
for i in range(10):
    data_[i] = np.random.random()

In [10]:
data_

{0: 0.11508530687678886,
 1: 0.05227020923955683,
 2: 0.2130711853254048,
 3: 0.3074397382829177,
 4: 0.9857577667059187,
 5: 0.24045758606641654,
 6: 0.9566574985890073,
 7: 0.5022385528218152,
 8: 0.9930012076449223,
 9: 0.7073703104407921}

In [71]:
split_dict(data_, (0.4, 0.4, 0.2))

[{8: 0.9930012076449223,
  5: 0.24045758606641654,
  3: 0.3074397382829177,
  4: 0.9857577667059187,
  6: 0.9566574985890073},
 {2: 0.2130711853254048,
  7: 0.5022385528218152,
  0: 0.11508530687678886,
  1: 0.05227020923955683},
 {9: 0.7073703104407921}]